Deliverable 2

In [1]:
# Deliverable 2: Determine Bias of Vine Reviews 
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,546 kB]
Get:13 http://se

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-09-30 03:32:53--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.97MB/s    in 0.2s    

2022-09-30 03:32:53 (4.97 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("vine_review_analysis").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

Load Amazon Data into Spark DataFrame

In [6]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Shoes_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18069663|R3P2HIOQCIN5ZU|B000XB31C0|     265024781|Minnetonka Men's ...|           Shoes|          1|            0|          0|   N|                Y|                   .|Do not buy: reall...| 2015-08-31|
|         US|   16251825|R12VVR0WH5Q24V|B00CFYZH5W|     259035853|Teva Men's Pajaro...|           Shoes|          5|    

Create Vine_Table DataFrames 

In [7]:
# Create the vine_table DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3P2HIOQCIN5ZU|          1|            0|          0|   N|                Y|
|R12VVR0WH5Q24V|          5|            0|          0|   N|                Y|
| RNCCKB6TV5EEF|          4|            0|          0|   N|                Y|
|R2NZXYIVCGB13W|          5|            0|          6|   N|                Y|
|R2EQ1TG9IT3OEQ|          3|            0|          0|   N|                Y|
|R1WXA9JSC2H1U1|          5|            1|          1|   N|                Y|
|R12ENYLFGGNWRV|          5|            1|          1|   N|                Y|
|R2R07E5PNXEUO5|          4|            0|          0|   N|                Y|
|R27BA52AKWMWN3|          5|            0|          0|   N|                Y|
| RLF8DOID2KD5O|          3|            0|          0|   N|     

In [8]:
# Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 20 to pick  
new_vine_df = vine_df.filter('total_votes>=20')
new_vine_df.show(truncate=False)

+--------------+-----------+-------------+-----------+----+-----------------+
|review_id     |star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R37F42INKX7L9K|5          |45           |49         |N   |Y                |
|R2EHKYNEP8WVSR|5          |25           |25         |N   |Y                |
|RXOS7BHID0UHL |5          |16           |27         |N   |N                |
|R33HHGFPB403GM|5          |19           |21         |N   |Y                |
|RY9O9XNLP464N |2          |19           |22         |N   |Y                |
|R2VP11C28JAEZF|5          |30           |30         |N   |Y                |
|R1TXGR1HA2M3P3|5          |28           |29         |N   |Y                |
|R6OD85TMEMHQQ |5          |28           |28         |N   |Y                |
|R1G4PAJXP3FTN7|2          |43           |51         |N   |Y                |
|R2P2S8UGUMCOLX|5          |21           |22         |N   |Y    

In [24]:
# The data is filtered to create a DataFrame or table where the percentage of helpful_votes is equal to or greater than 50% 
helpful_vine_df = new_vine_df.filter("helpful_votes/total_votes>=.50")
helpful_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R37F42INKX7L9K|          5|           45|         49|   N|                Y|
|R2EHKYNEP8WVSR|          5|           25|         25|   N|                Y|
| RXOS7BHID0UHL|          5|           16|         27|   N|                N|
|R33HHGFPB403GM|          5|           19|         21|   N|                Y|
| RY9O9XNLP464N|          2|           19|         22|   N|                Y|
|R2VP11C28JAEZF|          5|           30|         30|   N|                Y|
|R1TXGR1HA2M3P3|          5|           28|         29|   N|                Y|
| R6OD85TMEMHQQ|          5|           28|         28|   N|                Y|
|R1G4PAJXP3FTN7|          2|           43|         51|   N|                Y|
|R2P2S8UGUMCOLX|          5|           21|         22|   N|     

In [10]:
#Filter the DataFrame or table created in Step 2, and create a new DataFrame or table that retrieves all the rows where a review was written as part of 
#the Vine program (paid), vine == 'Y'.
# Load in a sql function to use columns 
from pyspark.sql.functions import col

# Filter last helpful_vine_y_df to only columns with vine == "Y" (Paid)
helpful_vine_y_df = helpful_vine_df.filter(col("vine") == "Y")
helpful_vine_y_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2N45ZKRRZS856|          5|           21|         22|   Y|                N|
| R5OMLMK13A8NS|          5|           34|         38|   Y|                N|
|R2MPEQ4SPTEQNS|          4|          180|        184|   Y|                N|
| RIR0D3KJ0CQ31|          4|           21|         21|   Y|                N|
|R1SPWJDHUWWC5E|          5|           88|         98|   Y|                N|
|R1X6M5XA3FT98W|          5|           24|         26|   Y|                N|
|R37VCW6HA0Z72T|          5|           27|         28|   Y|                N|
|R2XRYNV2SY3ZKL|          5|           53|         56|   Y|                N|
|R1Y93KWKAX1P5N|          2|           26|         31|   Y|                N|
|R2QVTDYYLTP8SL|          5|           21|         24|   Y|     

In [11]:
#Repeat Step 3, but this time retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'.
helpful_vine_n_df = helpful_vine_df.filter(col("vine") == "N")
helpful_vine_n_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R37F42INKX7L9K|          5|           45|         49|   N|                Y|
|R2EHKYNEP8WVSR|          5|           25|         25|   N|                Y|
| RXOS7BHID0UHL|          5|           16|         27|   N|                N|
|R33HHGFPB403GM|          5|           19|         21|   N|                Y|
| RY9O9XNLP464N|          2|           19|         22|   N|                Y|
|R2VP11C28JAEZF|          5|           30|         30|   N|                Y|
|R1TXGR1HA2M3P3|          5|           28|         29|   N|                Y|
| R6OD85TMEMHQQ|          5|           28|         28|   N|                Y|
|R1G4PAJXP3FTN7|          2|           43|         51|   N|                Y|
|R2P2S8UGUMCOLX|          5|           21|         22|   N|     

In [12]:
#Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid).
helpful_vine_df.count()

27009

In [13]:
# Determine total number of review - paid
helpful_vine_y_df.count()

22

In [14]:
# Determine total number of review - unpaid
helpful_vine_n_df.count()

26987

In [15]:
# Determine the number of reviews - 5-star 
helpful_vine_df.filter(col("star_rating") == 5).count()

14488

In [16]:
# Determine the number of reviews - 5-star paid 
helpful_vine_y_df.filter(col("star_rating") == 5).count()

13

In [17]:
# Determine the number of reviews - 5-star unpaid 
helpful_vine_n_df.filter(col("star_rating") == 5).count()

14475

In [20]:
# Determine percentage of Vine reviews that were 5 stars
first = helpful_vine_y_df.count()
second = helpful_vine_y_df.filter(col("star_rating") == 5).count()
percentage_paid = second/first * 100
print(str(round(percentage_paid,2)) + "% of paid reviews are 5-stars")

59.09% of paid reviews are 5-stars


In [28]:
# Determine percentage of non-Vine reviews that were 5 stars
firstnonvine = helpful_vine_n_df.count()
secondnonvine = helpful_vine_n_df.filter(col("star_rating") == 5).count()
percentage_unpaid = secondnonvine/firstnonvine * 100
print(str(round(percentage_unpaid,2)) + "% of unpaid reviews are 5-stars")

53.64% of unpaid reviews are 5-stars
